# RTE. Извлечение фичей из текстовых описаний

<br><br><br>
# Подготовка к работе

In [1]:
# 4COLAB: подключаем гугл-диск
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
# 4COLAB: установим отсутствующие модули
!pip install pymorphy2
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 86.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=f90670ab207fed2628e126e5ec093bcee531c0c7eac6f59394176141c4efdbc3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.2 MB/s eta 0:00:00


In [3]:
# импортируем необходимые модули
import pandas as pd
import numpy as np
import time
import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords

from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

import gensim

import torch
from transformers import AutoTokenizer, AutoModel

import warnings

In [4]:
# отключаем предупреждения
warnings.filterwarnings('ignore')

In [5]:
# настроим вывод датафреймов на экран
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 50)

In [6]:
# скачаем стоп-слова из nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# размеры векторов
W2V_SIZE = 300
BERT_SIZE = 312

# максимальное количество токенов BERT
BERT_MAX_LENGTH = 312  # максимум 2048, но долго

# путь до рабочей папки
OZON_PATH = '/content/drive/MyDrive/Colab Notebooks/Ozon/'

In [8]:
# функция возвращает разницу во времени в виде строки '(**m **s)' или '(<1s)'
def get_time(t1, t2):
    dt = t2 - t1
    if int(dt):
        return '(' + (f'{dt//60:.0f}m ' if dt//60 else '') + f'{dt%60:.0f}s)'
    return '(<1s)'

In [9]:
# функция запуска таймера
def timer_start(msg):
    print(msg, end='')
    return time.time()

In [10]:
# функция остановка таймера
def timer_stop(start_time):
    print(f'Готово! {get_time(start_time, time.time())}')

<br><br><br>
# Загрузка датафрейма

In [11]:
%%time
# загрузим данные в датафрейм `df`
df = pd.read_parquet(OZON_PATH + 'datasets/data_hakaton_2_5.parquet')

# переименуем столбцы и расположим в правильном порядке
df.columns = ['txt1', 'imgv1', 'imgv2', 'txt2', 'same']
df = df[['txt1', 'txt2', 'imgv1', 'imgv2', 'same']]

CPU times: user 5.25 s, sys: 2.68 s, total: 7.93 s
Wall time: 27.2 s


In [12]:
# df0 = df.copy()

In [13]:
# # возьмем часть от датафрейма
# fr = 0.001

# # соблюдаем изначальную нумерацию строк и баланс классов (с индекса 31088 начинается класс 0)
# df = pd.concat([df0[:int(31088*fr)], df0[31088:31088+int(31088*fr)]])

In [14]:
# out
print(df.shape)
df.head(1)

(62176, 5)


,txt1,txt2,imgv1,imgv2,same
0,"Зарядный кабель Borofone BX1 Lightning для айфон, 1м Зарядный кабель Borofone BX1 подходит для зарядки всех гаджетов и аксессуаров с разъемом Lightning. Поддерживает быструю зарядку. Подходит для ...","кабель проникновения производительность Borofone Lightning для айфон, 1м Зарядный кабель техника Borofone BX1 найти.v подходит Силикон гаджетов и аксессуаров с разъемом Lightning. Поддерживает быс...","[0.071516864, 1.097235, 0.1598482, 0.0, 1.2093426, 0.029174754, 1.1688348, 2.4246013, 0.0, 0.009662722, 0.0, 2.7528465, 0.08303496, 0.0026721905, 0.11616977, 0.17895052, 0.0096440585, 0.7796725, 0...","[0.0, 1.0745188, 0.0069836862, 0.0, 1.5283647, 0.0, 3.236915, 0.9076446, 0.0, 0.044373196, 0.5687515, 3.0410533, 0.0083197225, 0.7042634, 0.74028045, 0.2894601, 0.009714443, 1.33844, 0.0072414633,...",1


<br><br><br>
# Обработка текстовых данных

In [15]:
# функция возвращает очищенный текст (только русские, англ. буквы и одиночные пробелы)
def clean(text):
    text = re.sub(r'\</?[^\>]+\>', ' ', text)                               # html-теги
    text = re.sub(r'''(?:http[s]?://|www\.)(?:(?!"|'|\s).)+''', ' ', text)  # веб-ссылки
    text = re.sub(r'&[#\w]+;', ' ', text)                                   # спец-символы html
    text = text.lower()
    text = text.replace('ё', 'е')
    text = re.sub(r'[^a-zа-я]', ' ', text)    # все символы, кроме русских и англ. букв
    text = re.sub(r'\b\w{1,2}\b', ' ', text)  # слова в 1-2 символа
    text = re.sub(r'\s{2,}', ' ', text)       # более одного пробела
    text = text.strip(' ')
    return text

In [16]:
# функция возвращает словарь лемм для всех слов в датафрейме
def get_lemms(txt_series1, txt_series2):
    morph = MorphAnalyzer()
    # стоп-слова
    stopwords = nltk_stopwords.words('russian') + nltk_stopwords.words('english')
    # множество всех слов в обоих столбцах, за исключением стоп-слов
    words = set(' '.join(set(txt_series1).union(set(txt_series2))).split()) - set(stopwords)
    # возвращаем словарь {слово: лемма}
    return {word: morph.normal_forms(word)[0].replace('ё', 'е') for word in words}

In [17]:
# функция лемматизации текста
def lemmatize(text):
    return ' '.join([lemms[word] for word in text.split() if word in lemms])

In [18]:
# словарь соответствия POS-тегов pymorphy2 и word2vec
POS_TAGS = {'ADJF': 'ADJ', 'ADJS': 'ADJ', 'ADVB': 'ADV', 'COMP': 'ADV', 'PRED': 'ADV', 
            'INTJ': 'INTJ', 'NOUN': 'NOUN', 'NPRO': 'NOUN', 'NUMR': 'NUM', 
            'GRND': 'VERB', 'INFN': 'VERB', 'PRTF': 'VERB', 'PRTS': 'VERB', 'VERB': 'VERB', 
            'CONJ': 'X', 'PRCL': 'X', 'PREP': 'X'}

In [19]:
# функция возвращает POS-тег
def get_pos_tag(word, morph):
    pos_pymorphy2 = morph.parse(word)[0].tag.POS
    if pos_pymorphy2 in POS_TAGS:
        return POS_TAGS[pos_pymorphy2]
    return 'X'

In [20]:
# функция возвращает словарь pos-тегов для всех лемм из словаря lemms
def get_pos_tags(lemms_dct):
    morph = MorphAnalyzer()
    return {lemm: get_pos_tag(lemm, morph) for lemm in set(lemms_dct.values())}

In [21]:
# функция POS-тегирования слов (лемм) в тексте
def pos_tagging(text):
    return ' '.join([word + '_' + pos_tags[word] for word in text.split()])

<br>

---
#### Обработка текстовых данных

---


In [22]:
start = timer_start('Очистка текстовых данных... ')
for col in ['txt1', 'txt2']:
    df[col+'_lemm'] = df[col].apply(clean)
timer_stop(start)

start = timer_start('Подготовка множества всех слов и словаря лемм для каждого слова... ')
lemms = get_lemms(df['txt1_lemm'], df['txt2_lemm'])
timer_stop(start)

start = timer_start('Лемматизация... ')
for col in ['txt1_lemm', 'txt2_lemm']:
    df[col] = df[col].apply(lambda x: lemmatize(x))
timer_stop(start)

start = timer_start('Подготовка словаря POS-тегов для каждой леммы... ')
pos_tags = get_pos_tags(lemms)
timer_stop(start)

start = timer_start('POS-тегирование текстов... ')
for col in ['txt1_lemm', 'txt2_lemm']:
    df[col[:5]+'pos'] = df[col].apply(pos_tagging)
timer_stop(start)

Очистка текстовых данных... Готово! (19s)
Подготовка множества всех слов и словаря лемм для каждого слова... Готово! (19s)
Лемматизация... Готово! (4s)
Подготовка словаря POS-тегов для каждой леммы... Готово! (6s)
POS-тегирование текстов... Готово! (3s)


<br><br><br>
# Вычисление признаков

---
#### Функции: BERT-фичи

---


In [23]:
# функция вычисления BERT эмбеддингов
def get_bert_embeddings_tensor(text):
    tk = tokenizer(text, max_length=BERT_MAX_LENGTH, truncation=True, padding='max_length', 
                   add_special_tokens=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in tk.items()})
    embeddings = model_output.last_hidden_state
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [24]:
# функция возвращает массив векторов эмбеддингов слов из BERT-тензора (склеивает токены частей слов)
def get_words_embeddings(text, embeddings):
    # список токенов
    tokens = ['[CLS]'] + tokenizer.tokenize(text)[:BERT_MAX_LENGTH-2] + ['[SEP]']

    # # словарь соответствий токенов и эмбеддингов (раскомментировать для отладки)
    # print('tokens:', tokens)
    # print('len(tokens) =', len(tokens))
    # token_embeddings = {}
    # for i in range(len(tokens)):
    #     token_embeddings[tokens[i]] = embeddings[i]
    #     print(f'i={i:<3}  {tokens[i]:14s}  {token_embeddings[tokens[i]][:3]}')
    # print()
    # k = 0  # для распечатки word_embeddings

    # склеиваем токены частей слов в целые слова и получаем соответствующие им эмбеддинги (среднее всех частей слова)
    words_embeddings = []
    i = 1                       # исключаем нулевой индекс ([CLS])
    while i < len(tokens) - 1:  # исключаем последний индекс ([SEP])
        if tokens[i][:2] == '##':
            i += 1
        else:
            j = i + 1
            while j < len(tokens) - 1 and tokens[j][:2] == '##':
                j += 1
            words_embeddings.append(np.mean(embeddings[i:j], axis=0))
            # # распечатка эмбеддингов склеенных слов (раскомментировать для отладки)
            # print(f"k={k:<3} ", 
            #       f"{''.join((''.join(tokens[i:j])).split('##')):14s} ", 
            #       f"{np.mean(embeddings[i:j], axis=0)[:3]}")
            # k += 1
            i = j
    
    # возвращаем список векторов эмбеддингов слов
    return words_embeddings

In [25]:
# функция вычисляет EMDV для BERT-векторов слов
def get_bert_emdv(words_vecs1, words_vecs2):
    vec_s1 = get_semantic_info(words_vecs1, BERT_SIZE)
    vec_s2 = get_semantic_info(words_vecs2, BERT_SIZE)
    return abs(vec_s2 - vec_s1)

In [26]:
# функция возвращает кортеж из BERT-фичей
def bert_features(text1, text2):
    # тензоры BERT
    embeddings1 = get_bert_embeddings_tensor(text1)
    embeddings2 = get_bert_embeddings_tensor(text2)

    # эмбеддинги текстов
    bert1 = embeddings1[0]
    bert2 = embeddings2[0]

    # косинусное сходство между векторами bert1 и bert2
    bert_cos = cosine_similarity(bert1.reshape(1, -1), bert2.reshape(1, -1))[0][0]

    # списки эмбеддингов слов
    words_vecs1 = get_words_embeddings(text1, embeddings1)
    words_vecs2 = get_words_embeddings(text2, embeddings2)

    # эмбеддинги текстов, как поэлементные суммы эмбеддингов слов
    wbert1 = np.sum(np.array(words_vecs1), axis=0) if words_vecs1 else np.zeros(BERT_SIZE, dtype='float32')
    wbert2 = np.sum(np.array(words_vecs2), axis=0) if words_vecs2 else np.zeros(BERT_SIZE, dtype='float32')

    # косинусное сходство между векторами wbert1 и wbert2
    bert_sts = cosine_similarity(wbert1.reshape(1, -1), wbert2.reshape(1, -1))[0][0]

    # # EMDV для BERT-векторов слов  (закомментированы - не показали хорошее качество)
    # bert_emdv = get_bert_emdv(words_vecs1, words_vecs2)
    # bert_emdv_mean = bert_emdv.mean()

    return (bert1.tolist(), 
            bert2.tolist(), 
            bert_cos, 
            wbert1.tolist(), 
            wbert2.tolist(), 
            bert_sts)  # , bert_emdv.tolist(), bert_emdv_mean)

<br><br>

---
#### Функции: Поэлементный вектор Манхэттенских расстояний (EMDV)

---


In [27]:
# функция возвращает список w2v-векторов слов
def get_w2v_vectors(text_pos):
    return [w2v_model[word] for word in text_pos.split() if word in w2v_model]

In [28]:
# функция возвращает семантический вектор текста (векторы слов добавляются в него поэлементно, по порогу)
def get_semantic_info(words_vecs, vec_size):
    # если пустой список векторов, возвращаем массив нулей размером vec_size
    if not words_vecs:
        return np.zeros(vec_size, dtype='float32')
    vec_s = words_vecs[0].copy()
    for i in range(1, len(words_vecs)):
        x = words_vecs[i]
        alpha = x.mean() + x.std()
        for k in range(vec_size):
            if abs(vec_s[k] - x[k]) >= alpha:
                vec_s[k] += x[k]
    return vec_s

In [29]:
# функция вычисляет EMDV для w2v-векторов слов
def get_emdv(text1_pos, text2_pos):
    words_vecs1 = get_w2v_vectors(text1_pos)
    words_vecs2 = get_w2v_vectors(text2_pos)
    vec_s1 = get_semantic_info(words_vecs1, W2V_SIZE)
    vec_s2 = get_semantic_info(words_vecs2, W2V_SIZE)
    return abs(vec_s2 - vec_s1)

<br><br>

---
#### Функции: Сходство по Жаккару

---


In [30]:
# функция вычисляет сходство по Жаккару
def jaccar(text1, text2):
    t1 = set(text1.split())
    t2 = set(text2.split())
    jac = len(t1 & t2) / len(t1 | t2)
    return jac

<br><br>

---
#### Функции: Сходство на основе мешка слов

---


In [31]:
# функция рассчитывает косинусное сходство между двумя векторами из мешка слов
def bow_similarity(text1, text2):
    count_vect = CountVectorizer()
    corpus = [text1, text2]
    bow = count_vect.fit_transform(corpus)
    return cosine_similarity(bow[0], bow[1])[0][0]

<br><br>

---
#### Вычисление признаков

---


In [32]:
# загрузка предобученной BERT-модели
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

if torch.cuda.is_available():
    model.cuda()
model.device

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device(type='cuda', index=0)

In [33]:
start = timer_start('Вычисление BERT-фичей... ')
bert_series = df.apply(lambda row: bert_features(row['txt1_lemm'], row['txt2_lemm']), axis=1)
df[['bert1', 'bert2', 'bert_cos', 'wbert1', 'wbert2', 'bert_sts']] = pd.DataFrame(
    np.array(list(bert_series)), index=bert_series.index)  # , 'bert_emdv', 'bert_emdv_mean'
timer_stop(start)

start = timer_start('Рассчитываем косинусное сходство между векторами эмбеддингов изображений... ')
df['imgv_cos'] = df.apply(
    lambda row: cosine_similarity(row['imgv1'].reshape(1, -1), row['imgv2'].reshape(1, -1))[0][0], axis=1)
timer_stop(start)

start = timer_start('Загрузка предобученной w2v-модели для расчёта emdv... ')
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(
    OZON_PATH + 'rte/ruwikiruscorpora_upos_skipgram_300_2_2019.bin', binary=True)
# w2v_model.vector_size  # размер выходного вектора 300
timer_stop(start)

start = timer_start('Рассчитываем поэлементный вектор расстояний Манхэттена (EMDV)... ')
df['emdv'] = df.apply(lambda row: get_emdv(row['txt1_pos'], row['txt2_pos']), axis=1)
timer_stop(start)

start = timer_start('Рассчитываем среднее значение EMDV... ')
df['emdv_mean'] = df['emdv'].apply(lambda x: x.mean())
timer_stop(start)

start = timer_start('Рассчитываем сходство по Жаккару... ')
df['jac'] = df.apply(lambda row: jaccar(row['txt1_lemm'], row['txt2_lemm']), axis=1)
timer_stop(start)

start = timer_start('Рассчитываем сходство на основе мешка слов... ')
df['bow'] = df.apply(lambda row: bow_similarity(row['txt1_lemm'], row['txt2_lemm']), axis=1)
timer_stop(start)

Вычисление BERT-фичей... Готово! (16m 10s)
Рассчитываем косинусное сходство между векторами эмбеддингов изображений... Готово! (25s)
Загрузка предобученной w2v-модели для расчёта emdv... Готово! (7s)
Рассчитываем поэлементный вектор расстояний Манхэттена (EMDV)... Готово! (31m 29s)
Рассчитываем среднее значение EMDV... Готово! (<1s)
Рассчитываем сходство по Жаккару... Готово! (3s)
Рассчитываем сходство на основе мешка слов... Готово! (2m 10s)


<br><br><br>
# Сохранение результатов

In [34]:
# запись датафрейма, часть вторая, полная версия, со всеми столбцами и фичами, включая векторы картинок и BERT-фичи
df.to_parquet(OZON_PATH + 'datasets/ds_2_5_features_all.parquet')

# запись датафрейма, всё, кроме эмбеддингов изображений
df[['txt1', 'txt2', 'txt1_lemm', 'txt2_lemm', 'txt1_pos', 'txt2_pos', 
    'imgv_cos', 'emdv', 'emdv_mean', 'jac', 'bow', 
    'bert1', 'bert2', 'bert_cos', 'wbert1', 'wbert2', 'bert_sts', 'same']].to_parquet(
        OZON_PATH + 'datasets/ds_2_5_features_all_except_imgv.parquet')

# запись датафрейма только со скалярными фичами изображений и текстов
df[['txt1', 'txt2', 'txt1_lemm', 'txt2_lemm', 'txt1_pos', 'txt2_pos', 
    'imgv_cos', 'emdv_mean', 'jac', 'bow', 'bert_cos', 'bert_sts', 'same']].to_parquet(
        OZON_PATH + 'datasets/ds_2_5_features_scalar.parquet')

# запись датафрейма, только 5 фичей метода RTE из статьи, для бейзлайна мультимодальной модели
df[['emdv', 'emdv_mean', 'jac', 'bow', 'bert_sts']].to_parquet(
        OZON_PATH + 'datasets/ds_2_5_rte_five_features_312_tokens.parquet')